In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from rfpimp import importances
%matplotlib inline
pd.set_option("display.max_columns", 2000)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
df_full=pd.read_csv('data/Seasons_Stats.csv')
df_full.drop(columns='Unnamed: 0', inplace=True)

In [20]:
df=df_full[df_full['Year']>=2005]
df.drop(labels='blank2', axis=1, inplace=True)
df.drop(labels='blanl', axis=1, inplace=True)
df['Year']=df['Year'].astype(int)
df['Year']=pd.to_datetime(df['Year'], format='%Y')
df['Year']=df['Year'].dt.year

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [33]:
df.head(10)

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
17076,2005,Shareef Abdur-Rahim,PF,28.0,POR,54.0,49.0,1867.0,18.7,0.581,0.058,0.379,7.9,16.0,12.1,10.7,1.4,1.0,13.0,22.1,3.9,1.5,5.4,0.139,1.4,-0.5,0.8,1.3,337.0,670.0,0.503,15.0,39.0,0.385,322.0,631.0,0.510,0.514,220.0,254.0,0.866,123.0,269.0,392.0,111.0,49.0,26.0,117.0,150.0,909.0
17077,2005,Cory Alexander,SG,31.0,CHA,16.0,1.0,201.0,8.8,0.451,0.388,0.245,4.3,12.6,8.2,27.7,2.3,0.4,25.9,16.2,-0.2,0.2,0.0,0.002,-2.9,-0.5,-3.4,-0.1,16.0,49.0,0.327,8.0,19.0,0.421,8.0,30.0,0.267,0.408,9.0,12.0,0.750,8.0,21.0,29.0,37.0,9.0,1.0,19.0,29.0,49.0
17078,2005,Malik Allen,PF,26.0,TOT,36.0,1.0,519.0,14.2,0.512,0.000,0.158,9.0,13.3,11.0,5.7,0.9,3.2,8.7,18.1,0.6,0.5,1.1,0.100,-2.0,-0.3,-2.3,0.0,84.0,177.0,0.475,0.0,0.0,NaN,84.0,177.0,0.475,0.475,26.0,28.0,0.929,40.0,59.0,99.0,18.0,9.0,22.0,18.0,53.0,194.0
17079,2005,Malik Allen,PF,26.0,MIA,14.0,0.0,248.0,13.6,0.505,0.000,0.184,11.8,12.7,12.2,7.0,0.9,3.3,11.8,17.5,0.3,0.3,0.6,0.113,-1.7,0.6,-1.1,0.1,35.0,76.0,0.461,0.0,0.0,NaN,35.0,76.0,0.461,0.461,13.0,14.0,0.929,24.0,28.0,52.0,11.0,4.0,11.0,11.0,29.0,83.0
17080,2005,Malik Allen,PF,26.0,CHA,22.0,1.0,271.0,14.8,0.518,0.000,0.139,6.4,13.8,9.9,4.6,1.0,3.1,6.1,18.7,0.3,0.2,0.5,0.088,-2.4,-1.0,-3.4,-0.1,49.0,101.0,0.485,0.0,0.0,NaN,49.0,101.0,0.485,0.485,13.0,14.0,0.929,16.0,31.0,47.0,7.0,5.0,11.0,7.0,24.0,111.0
17081,2005,Ray Allen,SG,29.0,SEA,78.0,78.0,3064.0,20.9,0.555,0.372,0.286,3.2,10.7,6.9,18.1,1.5,0.1,9.2,28.0,9.7,1.0,10.7,0.168,5.6,-1.9,3.7,4.4,640.0,1494.0,0.428,209.0,556.0,0.376,431.0,938.0,0.459,0.498,378.0,428.0,0.883,79.0,268.0,347.0,289.0,84.0,5.0,171.0,167.0,1867.0
17082,2005,Tony Allen,SG,23.0,BOS,77.0,34.0,1262.0,14.7,0.542,0.080,0.393,8.0,12.3,10.2,8.2,3.1,1.4,14.5,18.9,1.2,1.6,2.7,0.103,-0.7,0.6,-0.1,0.6,184.0,387.0,0.475,12.0,31.0,0.387,172.0,356.0,0.483,0.491,112.0,152.0,0.737,85.0,136.0,221.0,64.0,76.0,24.0,77.0,156.0,492.0
17083,2005,Rafer Alston,PG,28.0,TOR,80.0,78.0,2717.0,16.5,0.522,0.399,0.265,1.7,10.1,5.8,31.7,2.2,0.2,13.5,21.2,3.8,1.7,5.5,0.098,2.6,-1.2,1.5,2.4,403.0,974.0,0.414,139.0,389.0,0.357,264.0,585.0,0.451,0.485,191.0,258.0,0.740,42.0,237.0,279.0,514.0,118.0,7.0,170.0,212.0,1136.0
17084,2005,Chris Andersen,C,26.0,NOH,67.0,2.0,1430.0,18.5,0.581,0.008,0.531,10.9,23.6,17.0,9.1,0.5,5.6,12.7,16.4,2.7,1.8,4.5,0.152,-1.1,2.4,1.2,1.2,191.0,358.0,0.534,0.0,3.0,0.000,191.0,355.0,0.538,0.534,131.0,190.0,0.689,137.0,273.0,410.0,71.0,14.0,100.0,64.0,144.0,513.0
17085,2005,Derek Anderson,SG,30.0,POR,47.0,32.0,1239.0,11.7,0.503,0.386,0.223,2.4,9.2,6.0,18.9,1.6,0.2,14.0,18.5,0.7,0.6,1.3,0.052,-0.3,-1.8,-2.1,0.0,152.0,391.0,0.389,58.0,151.0,0.384,94.0,240.0,0.392,0.463,70.0,87.0,0.805,25.0,103.0,128.0,143.0,36.0,4.0,70.0,89.0,432.0


In [36]:
def remove_tot(combined):
    tot_player=combined[combined['Tm']=='TOT']['Player'].values
    for name in tot_player:
        temp=combined[combined['Player']==name]
        tot_index=temp[temp['Tm']=='TOT'].index.values
        for idx in tot_index:
            year=temp.loc[idx]['Year']
            year_index=temp[temp['Year']==year].index.values
            year_temp=temp[temp['Year']==year]
            max_year=((year_temp[year_temp['Tm']!='TOT'])['G'].max())
            team=(year_temp[year_temp['G']==max_year])['Tm'].values
            team=str(team)
            team=team[1:-1]
            combined.at[idx,'Tm']=team
            for element in year_index:
                if element==idx:
                    continue
                else:
                    combined.drop(axis=0, index=element, inplace=True)
    return(combined)

In [37]:
df=remove_tot(df)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [44]:
# dropping player with the same name:
#df[df.duplicated(subset=['Year','Player'])]
df=df.drop_duplicates(subset=['Year','Player'], keep=False)

# Selecting the features for the model:


In [ ]:
#list of features I assume help predict player performance
# features used for prediction are past year data
# Features:
    #prev_ppg
    #prev_min
    #current age
    #prev_rebound
    #prev block
    #prev steal
    #prev turnover